In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
ticker = yf.Ticker("AAPL")
info = ticker.info
 
data = yf.download("IBM", start='2020-10-01', end='2023-10-01')
df = pd.DataFrame(data)
df['SMA_10'] = df['Close'].rolling(window=10).mean()
df.dropna(subset=['SMA_10'], inplace=True)
df.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [16]:
# Defino dos decimales para los datos flotantes
pd.options.display.float_format = '{:.2f}'.format
df.head(15)

,Date,Open,High,Low,Close,Adj Close,Volume,SMA_10
0,2020-10-14,119.63,121.36,119.63,120.40,103.73,3899593,119.24
1,2020-10-15,118.62,119.71,118.40,119.40,102.86,3541024,119.60
2,2020-10-16,119.67,120.87,119.18,120.39,103.72,4927078,120.11
3,2020-10-19,121.22,121.75,119.58,120.00,103.38,7822720,120.45
4,2020-10-20,114.53,114.87,111.70,112.21,96.67,22490151,120.01
5,2020-10-21,111.53,112.51,109.74,110.00,94.76,10204044,119.15
6,2020-10-22,109.94,110.96,108.01,110.67,95.34,8217167,117.65
7,2020-10-23,111.38,111.49,110.45,110.90,95.54,4072496,116.52
8,2020-10-26,109.42,109.85,106.92,107.28,92.43,7534756,115.09
9,2020-10-27,107.22,107.28,105.19,105.70,91.06,6209161,113.70


In [17]:
df.corr()

,Date,Open,High,Low,Close,Adj Close,Volume,SMA_10
Date,1.00,0.52,0.51,0.52,0.51,0.79,-0.18,0.53
Open,0.52,1.00,0.99,0.99,0.99,0.92,-0.21,0.95
High,0.51,0.99,1.00,0.99,0.99,0.92,-0.19,0.95
Low,0.52,0.99,0.99,1.00,1.00,0.93,-0.24,0.95
Close,0.51,0.99,0.99,1.00,1.00,0.93,-0.22,0.94
Adj Close,0.79,0.92,0.92,0.93,0.93,1.00,-0.24,0.89
Volume,-0.18,-0.21,-0.19,-0.24,-0.22,-0.24,1.00,-0.16
SMA_10,0.53,0.95,0.95,0.95,0.94,0.89,-0.16,1.00


In [33]:
periodo_atr = 14
df['Rango'] = df['High'] - df['Low']
df['TR'] = df[['High', 'Close']].max(axis=1) - df[['Low', 'Close']].min(axis=1)
df['ATR'] = df['TR'].rolling(window=periodo_atr).mean()
df.iloc[30]

Date         2020-11-25 00:00:00
Open                      117.52
High                      118.86
Low                       116.74
Close                     118.74
Adj Close                 103.78
Volume                   4326151
SMA_10                    113.48
Rango                       2.12
TR                          2.12
ATR                         2.24
Name: 30, dtype: object

In [32]:
time_step = 30
data_in = []
data_out = []

for i in range(time_step, len(df)-time_step):
    data_in.append(df[i-time_step: i])
    data_out.append(df.iloc[i])

print(data_in[0].iloc[29])
print(data_out[0])

Date         2020-11-24 00:00:00
Open                      115.54
High                      119.24
Low                       115.50
Close                     118.95
Adj Close                 103.96
Volume                   8109115
SMA_10                    112.81
Rango                       3.75
TR                          3.75
ATR                         2.24
Name: 29, dtype: object
Date         2020-11-25 00:00:00
Open                      117.52
High                      118.86
Low                       116.74
Close                     118.74
Adj Close                 103.78
Volume                   4326151
SMA_10                    113.48
Rango                       2.12
TR                          2.12
ATR                         2.24
Name: 30, dtype: object
